# 15. ALZ Transfer Learning with VGG19 on ADNI images
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mcqueg/cnn-alzheimers/blob/main/notebooks/15-TL-vgg19.ipynb)  

In [ ]:
# mount drive for saving model checkpoints
from google.colab import drive
drive.mount('/content/gdrive')

# load github repo and installing requirements to train the model
!git clone https://github.com/mcqueg/cnn-alzheimers.git
!git config --global user.name "mcqueg"
!git config --global user.email "garrettmccue@gmail.com"
%cd cnn-alzheimers/
!pip install -r requirements.txt

In [ ]:
from src.models.model_utils import load_model
from src.models.train_val import train_val
import os
from datetime import datetime
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam


In [ ]:
# workspaces
COLAB = '/content/cnn-alzheimers/'
GDRIVE = '/content/gdrive/Othercomputers/MacBookPro/cnn-alzheimers/'

# load from colab after mounting repo
TRAIN_DIR = f'{COLAB}data/processed/ADNI/train'
TEST_DIR = f'{COLAB}data/processed/ADNI/test'
LOGS_DIR = f'{COLAB}/logs'

# save to GDRIVE
SAVE_DIR = f'{GDRIVE}models/checkpoints'
os.listdir(TRAIN_DIR)

In [ ]:
JSON_PATH = '/content/gdrive/Othercomputers/MacBookPro/cnn-alzheimers/models/checkpoints/inception-v3-alz-conv-mixed8__01-08-2022_00:23:13/config/inception-v3-alz-conv-mixed8__01-08-2022_00:23:13.json'
WEIGHTS_PATH = '/content/gdrive/Othercomputers/MacBookPro/cnn-alzheimers/models/checkpoints/inception-v3-alz-conv-mixed8__01-08-2022_00:23:13/ckpt/inception-v3-alz-conv-mixed8__01-08-2022_00:23:13_weights.h5'
LAST_FROZEN_LAYER = 'mixed10'
# TRAIN PARAMS:
LR = 0.001
DROP = 0.2
VAL_SIZE = 0.5 # splits test dir in half for  validation and testing data
EPOCHS = 500
BATCH = 16
CLASS_NUM = 5

MODEL_NAME = 'vgg19-tl-alz_adni'
print(f'saving model details under: {MODEL_NAME}')

In [ ]:
model = load_model(JSON_PATH,
                   WEIGHTS_PATH,
                   LAST_FROZEN_LAYER)
for layer in model.layers:
    layer.trainable = False

# set output of the conv layers
last_layer = model.get_layer(LAST_FROZEN_LAYER)
print(f'last layer output shape: {last_layer.output_shape}')
last_output = last_layer.output
# feed output into a fresh dense network

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# add a fully connected layer
x = layers.Dense(1024, activation='relu')(x)
# add the dropout rate of 0.2
x = layers.Dropout(DROP)(x)
# add a final layer with softmax for classification
x = layers.Dense(CLASS_NUM, activation='softmax')(x)

# append the created dense network to the pre_trained_model
model = Model(model.input, x)

# print model summary
model.summary()

In [ ]:
 # compile model
model.compile(optimizer = Adam(learning_rate=LR),
                  loss = 'categorical_crossentropy',
                  metrics = ['accuracy'])

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs --host 0.0.0.0

In [ ]:
history = train_val(model=model,
                    name=name,
                    weights_path=save_weights_path,
                    train_dir=TRAIN_DIR,
                    test_dir =TEST_DIR,
                    logs_dir=LOGS_DIR,
                    save_dir=SAVE_DIR,
                    val_size=VAL_SIZE,
                    epochs=EPOCHS,
                    lr=LR,
                    batch_size=BATCH,
                    evaluate=True)